### 10. Linear Regression

#### Table of Contents

- [1. Linear regression implementation with numpy](#heading)
- [2. Linear regression implementation with autograd](#heading)
- [3. Linear regression implementation with pytorch loss and optimizer](#heading)
- [4. Linear regression implementation with pytorch model](#heading)

#### 1. Linear regression implementation with numpy

 - Predition: manually
 - Gradients computation: manually
 - Loss computation: manually
 - Parameter updates: manually

In [1]:
import numpy as np

# linear regression function: f = w * x
# example: f = 2 * x (final weight w = 2)

x = np.array([1, 2, 3, 4], dtype=np.float32)
y = np.array([2, 4, 6, 8], dtype=np.float32)

w = 0.0 # initial weight

# model prediction
def forward(x):
    return w * x

# loss computation: loss = MSE
# loss = 1/N * (y_predicted - y)^2
def loss(y, y_predicted):
    return ((y_predicted-y)**2).mean()

# gradient computation
# MSE = 1/N * (wx - y)**2
# dJ/dw = 1/N * 2x * (wx-y)
def gradient(x, y, y_predicted):
    return np.dot(2*x, y_predicted-y).mean()

print('Prediction before training: f(x) = ', forward(x))
print(f'Prediction before training: f(5) = {forward(5):.3f}')

Prediction before training: f(x) =  [0. 0. 0. 0.]
Prediction before training: f(5) = 0.000


In [2]:
# training
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    # prediction, forward pass
    y_pred = forward(x)

    # loss computation
    l = loss(y, y_pred)

    # gradients
    dw = gradient(x, y, y_pred)

    # update weights: w = w - lr * dJ/dw
    w -= learning_rate * dw

    if epoch % 2 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {l:.8f}')

print('Prediction after training: f(x) = ', forward(x))
print(f'Prediction after training: f(5) = {forward(5):.3f}')

epoch 1: w = 1.200, loss = 30.00000000
epoch 3: w = 1.872, loss = 0.76800019
epoch 5: w = 1.980, loss = 0.01966083
epoch 7: w = 1.997, loss = 0.00050331
epoch 9: w = 1.999, loss = 0.00001288
epoch 11: w = 2.000, loss = 0.00000033
epoch 13: w = 2.000, loss = 0.00000001
epoch 15: w = 2.000, loss = 0.00000000
epoch 17: w = 2.000, loss = 0.00000000
epoch 19: w = 2.000, loss = 0.00000000
Prediction after training: f(x) =  [2. 4. 6. 8.]
Prediction after training: f(5) = 10.000


#### 2. Linear regression implementation with autograd

 - Prediction: manually
 - Gradients computation: **autograd**
 - Loss computation: manually
 - Parameter updates: manually

In [3]:
import torch

# linear regression function: f = w * x
# example: f = 2 * x (final weight w = 2)

x = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True) # initial weight

# model prediction
def forward(x):
    return w * x

# loss computation: loss = MSE
# loss = 1/N * (y_predicted - y)^2
def loss(y, y_predicted):
    return ((y_predicted-y)**2).mean()

# gradient computation
# MSE = 1/N * (wx - y)**2
# dJ/dw = 1/N * 2x * (wx-y)
# def gradient(x, y, y_predicted):
#     return np.dot(2*x, y_predicted-y).mean()

print('Prediction before training: f(x) = ', forward(x))
print(f'Prediction before training: f(5) = {forward(5):.3f}')

# training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    # prediction, forward pass
    y_pred = forward(x)

    # loss computation
    l = loss(y, y_pred)

    # gradients = backward pass
    l.backward() # dl/dw

    # update weights: w = w - lr * dJ/dw
    with torch.no_grad():
        w -= learning_rate * w.grad

    # zero gradients
    # you can comment out this line and check the training process
    w.grad.zero_()

    if epoch % 10 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {l:.8f}')

print('Prediction after training: f(x) = ', forward(x))
print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(x) =  tensor([0., 0., 0., 0.], grad_fn=<MulBackward0>)
Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 11: w = 1.665, loss = 1.16278565
epoch 21: w = 1.934, loss = 0.04506890
epoch 31: w = 1.987, loss = 0.00174685
epoch 41: w = 1.997, loss = 0.00006770
epoch 51: w = 1.999, loss = 0.00000262
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(x) =  tensor([2.0000, 4.0000, 6.0000, 8.0000], grad_fn=<MulBackward0>)
Prediction after training: f(5) = 10.000


#### 3. Linear regression implementation with pytorch loss and optimizer

 - Prediction: manually
 - Gradients computation: **autograd**
 - Loss computation: **pytorch loss**
 - Parameter updates: **pytorch optimizer**

Training pipeline in Pytorch has 3 steps:
 - 1. Design model (input size, output size, forward pass)
 - 2. Construct loss and optimizer
 - 3. Training loop, 3 steps:
  - forward pass: compute prediction
  - backward pass: gradients
  - update weights

In [4]:
import torch
import torch.nn as nn

# linear regression function: f = w * x
# example: f = 2 * x (final weight w = 2)

x = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True) # initial weight

# model prediction
def forward(x):
    return w * x

print('Prediction before training: f(x) = ', forward(x))
print(f'Prediction before training: f(5) = {forward(5):.3f}')

# training
learning_rate = 0.01
n_iters = 100

loss = nn.MSELoss()
optimizer = torch.optim.SGD([w], lr=learning_rate)

for epoch in range(n_iters):
    # prediction, forward pass
    y_pred = forward(x)

    # loss computation
    l = loss(y, y_pred)

    # gradients = backward pass
    l.backward() # dl/dw

    # update weights: w = w - lr * dJ/dw
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if epoch % 10 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {l:.8f}')

print('Prediction after training: f(x) = ', forward(x))
print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(x) =  tensor([0., 0., 0., 0.], grad_fn=<MulBackward0>)
Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 11: w = 1.665, loss = 1.16278565
epoch 21: w = 1.934, loss = 0.04506890
epoch 31: w = 1.987, loss = 0.00174685
epoch 41: w = 1.997, loss = 0.00006770
epoch 51: w = 1.999, loss = 0.00000262
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(x) =  tensor([2.0000, 4.0000, 6.0000, 8.0000], grad_fn=<MulBackward0>)
Prediction after training: f(5) = 10.000


#### 4. Linear regression implementation with pytorch model

 - Prediction: **pytorch model** (self-defined model class)
 - Gradients computation: **autograd**
 - Loss computation: **pytorch loss**
 - Parameter updates: **pytorch optimizer**

In [5]:
import torch
import torch.nn as nn

# linear regression function: f = w * x
# example: f = 2 * x (final weight w = 2)

x = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

x_test = torch.tensor([5], dtype=torch.float32)

n_samples, n_features = x.size()
input_size = n_features
output_size = n_features

# model prediction
model = nn.Linear(in_features=input_size, out_features=output_size, bias=False)

# print('model weight: ', model.weight)
# print('Prediction before training: f(x) = ', model(x))
print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')

# loss function and optimizer
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    # prediction, forward pass
    y_pred = model(x)

    # loss computation
    l = loss(y, y_pred)

    # gradients = backward pass
    l.backward() # dl/dw

    # update weights: w = w - lr * dJ/dw
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if epoch % 10 == 0:
        [w] = model.parameters()
        print(f'epoch {epoch+1}: w = {w.item():.3f}, loss = {l:.8f}')

# print('Prediction after training: f(x) = ', model(x))
print(f'Prediction after training: f(5) = {model(x_test).item():.3f}')

Prediction before training: f(5) = 0.823
epoch 1: w = 0.440, loss = 25.26521111
epoch 11: w = 1.693, loss = 0.97926784
epoch 21: w = 1.940, loss = 0.03795604
epoch 31: w = 1.988, loss = 0.00147113
epoch 41: w = 1.998, loss = 0.00005702
epoch 51: w = 2.000, loss = 0.00000221
epoch 61: w = 2.000, loss = 0.00000009
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(5) = 10.000


**Customized model class:**

In [6]:
import torch
import torch.nn as nn

# linear regression function: f = w * x
# example: f = 2 * x (final weight w = 2)

x = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

x_test = torch.tensor([5], dtype=torch.float32)

n_samples, n_features = x.size()
input_size = n_features
output_size = n_features

# customized model class for forward pass or prediction
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim, bias=False)

    def forward(self, x):
        return self.lin(x)

model = LinearRegression(input_dim=input_size, output_dim=output_size)

print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')

# loss function and optimizer
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    # prediction, forward pass
    y_pred = model(x)

    # loss computation
    l = loss(y, y_pred)

    # gradients = backward pass
    l.backward() # dl/dw

    # update weights: w = w - lr * dJ/dw
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if epoch % 10 == 0:
        [w] = model.parameters()
        print(f'epoch {epoch+1}: w = {w.item():.3f}, loss = {l:.8f}')

print(f'Prediction after training: f(5) = {model(x_test).item():.3f}')

Prediction before training: f(5) = -0.104
epoch 1: w = 0.282, loss = 30.62747955
epoch 11: w = 1.662, loss = 1.18710721
epoch 21: w = 1.933, loss = 0.04601177
epoch 31: w = 1.987, loss = 0.00178339
epoch 41: w = 1.997, loss = 0.00006912
epoch 51: w = 1.999, loss = 0.00000268
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(5) = 10.000
